<a href="https://colab.research.google.com/github/rahiakela/computer-vision-research-and-practice/blob/main/hands-on-computer-vision-with-detectron2/05-custom-object-detection/02_default_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Default_Trainer

Detectron2 provides a default trainer class, which helps to train Detectron2 models on custom datasets
conveniently

##Setup

In [ ]:
%%capture

!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"
#!sudo apt-get install tree
#!pip install -q pylabel

In [2]:
import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2 import model_zoo

import os
import pickle
import shutil
import random
import logging
import math
from glob import glob
from tqdm import tqdm
#import yaml
#from yaml.loader import SafeLoader
#from pylabel import importer

import torch
import cv2

import numpy as np
import matplotlib.pyplot as plt

from detectron2.utils.logger import setup_logger

# Suppress some user warnings
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
print(detectron2.__version__)

0.6


In [ ]:
!wget https://github.com/rahiakela/computer-vision-research-and-practice/blob/main/hands-on-computer-vision-with-detectron2/05-custom-object-detection/dataset/braintumors_coco.zip?raw=true -O braintumors_coco.zip
!unzip -q braintumors_coco.zip

In [5]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_train = name_ds + "_train"
name_ds_test = name_ds + "_test"
image_root_train = name_ds + "/train"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_train = name_ds + "/train/" + af
json_file_test = name_ds + "/test/" + af

# Register datasets
## train dataset
register_coco_instances(
    name=name_ds_train,
    metadata={},
    json_file=json_file_train,
    image_root=image_root_train
)
## test dataset
register_coco_instances(
    name=name_ds_test,
    metadata={},
    json_file=json_file_test,
    image_root=image_root_test
)

##Training configuration

In [6]:
# let's prepare a logger that Detectron2 uses to log training/inferencing information
logger = setup_logger()

In [7]:
# getting a training configuration
output_dir = "output/object_detector"
os.makedirs(output_dir, exist_ok=True)

output_cfg_path = os.path.join(output_dir, "cfg.pickle")

nc = 2
device = "cuda"

config_file_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [8]:
# let's create a configuration file
cfg = get_cfg()
config_file = model_zoo.get_config_file(config_file_url)
cfg.merge_from_file(config_file)

# Download weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)

# Set datasets
cfg.DATASETS.TRAIN = (name_ds_train,)
cfg.DATASETS.TEST = (name_ds_test,)

# Workers
cfg.DATALOADER.NUM_WORKERS = 2

# Images per batch
cfg.SOLVER.IMS_PER_BATCH = 4  # 8

# Learning rate
cfg.SOLVER.BASE_LR = 0.00025

# Iterations
cfg.SOLVER.MAX_ITER  = 5000
cfg.SOLVER.CHECKPOINT_PERIOD  = 500

# Classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = nc
cfg.MODEL.DEVICE = device

cfg.OUTPUT_DIR = output_dir

In [9]:
# save configuration file for future use
with open(output_cfg_path, "wb") as f:
  pickle.dump(cfg, f, protocol=pickle.HIGHEST_PROTOCOL)

##Model Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.train()

In [ ]:
# let's see the log events
!ls -1 {cfg.OUTPUT_DIR}

##Best model

##Displaying Samples

In [ ]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_train = name_ds + "_train"
name_ds_test = name_ds + "_test"
image_root_train = name_ds + "/train"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_train = name_ds + "/train/" + af
json_file_test = name_ds + "/test/" + af

# Register datasets
## train dataset
register_coco_instances(
    name=name_ds_train,
    metadata={},
    json_file=json_file_train,
    image_root=image_root_train
)
##test dataset
register_coco_instances(
    name=name_ds_test,
    metadata={},
    json_file=json_file_test,
    image_root=image_root_test
)

In [ ]:
random.seed(42)

def plot_random_samples(name_ds, n=3):
  # access
  dataset = DatasetCatalog.get(name_ds)
  metadata = MetadataCatalog.get(name_ds)
  # plot (arrange in 3x3)
  nrows = int(-(-n/3)) # ceil
  ncols = 3
  fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5))
  for i,s in enumerate(random.sample(dataset, n)):
    ax = axs[i//ncols][i%ncols] if len(axs.shape) == 2 else axs[i]
    img = cv2.imread(s["file_name"])
    v = Visualizer(img[:,:, ::-1], metadata=metadata, scale=0.5)
    v = v.draw_dataset_dict(s)
    ax.imshow(v.get_image())
    ax.axis("off")
  plt.tight_layout()
  plt.show()

In [ ]:
# let's displays three sample images and their corresponding labels from the train dataset
plot_random_samples(name_ds_train, n=3)

In [ ]:
# let's displays three sample images and their corresponding labels from the test dataset
plot_random_samples(name_ds_test, n=3)

In [ ]:
# zip converted dataset
!zip braintumors_coco.zip braintumors_coco/train/* braintumors_coco/test/*